In [ ]:
# de python, para especificar rutas de archivos y directorios
from pathlib import Path
import math
import datetime

# lib para trabajar con arrays
import numpy as np
import pandas as pd
import sklearn

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt
import seaborn as sns

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten, Rescaling, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential


# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

### AMBIENTE DE TEST Y TRAIN

In [ ]:
from keras.datasets import fashion_mnist

train, test = fashion_mnist.load_data()
(X_train, Y_train) = train 
(X_test, Y_test) = test


### Prueba usando iamge net red resnet50

In [ ]:
X_train = np.array(X_train).reshape(-1, 28, 28, 1)
X_test = np.array(X_test).reshape(-1, 28, 28, 1)

X_train.shape


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

def change_size(image):
    img = array_to_img(image, scale=False) #returns PIL Image
    img = img.resize((75, 75)) #resize image
    img = img.convert(mode='RGB') #makes 3 channels
    arr = img_to_array(img) #convert back to array
    return arr.astype(np.float64)

In [ ]:
train_arr_75 = [change_size(img) for img in X_train]
del X_train
train_arr_75 = np.array(train_arr_75)
train_arr_75.shape

In [ ]:
test_arr_75 = [change_size(img) for img in X_test]
del X_test
test_arr_75 = np.array(test_arr_75)
test_arr_75.shape

In [ ]:

image_gen = ImageDataGenerator(rescale=1./255, #easier for network to interpret numbers in range [0,1]
                              zoom_range=0.1,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              validation_split=0.2) # 80/20 train/val split

train_generator = image_gen.flow(train_arr_75, 
                                Y_train,
                                batch_size=32,
                                shuffle=True,
                                subset='training',
                                seed=42)
valid_generator = image_gen.flow(test_arr_75,
                                Y_test,
                                batch_size=16,
                                shuffle=True,
                                subset='validation')
del train_arr_75 #saves RAM


In [ ]:
model = Sequential()

model.add(tf.keras.applications.resnet50.ResNet50(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for layer in model.layers[0].layers:
    if layer.name == 'conv5_block1_0_conv':
        break
    layer.trainable=False

In [ ]:
history = model.fit(train_arr_75,
Y_train,
epochs = 5,
validation_data=(test_arr_75, Y_test),
verbose = 1)

'''history = model.fit(train_generator, validation_data=valid_generator, epochs=5, 
        steps_per_epoch=train_generator.n//train_generator.batch_size,
        validation_steps=valid_generator.n//valid_generator.batch_size)'''


### Prueba TL creando nosotros una red

In [ ]:
pt_x_train = []
pt_y_train = []
pt_x_test = []
pt_y_test = []

tl_x_train = []
tl_y_train = []
tl_x_test = []
tl_y_test = []

for i in range(len(X_train)):
    if Y_train[i] < 5:
        pt_x_train.append(X_train[i] / 255) # Normalize
        pt_y_train.append(Y_train[i])
    else:
        tl_x_train.append(X_train[i] / 255)
        tl_y_train.append(Y_train[i])
        
for i in range(len(X_test)):
    if Y_test[i] < 5:
        pt_x_test.append(X_test[i] / 255)
        pt_y_test.append(Y_test[i])
    else:
        tl_x_test.append(X_test[i] / 255)
        tl_y_test.append(Y_test[i])

In [ ]:


pt_x_train = np.asarray(pt_x_train).reshape(-1, 28, 28, 1)
pt_x_test = np.asarray(pt_x_test).reshape(-1, 28, 28, 1)
pt_y_train = np_utils.to_categorical(np.asarray(pt_y_train))
pt_y_test = np_utils.to_categorical(np.asarray(pt_y_test))

tl_x_train = np.asarray(tl_x_train).reshape(-1, 28, 28, 1)
tl_x_test = np.asarray(tl_x_test).reshape(-1, 28, 28, 1)
tl_y_train = np_utils.to_categorical(np.asarray(tl_y_train))
tl_y_test = np_utils.to_categorical(np.asarray(tl_y_test))

print("Pre Training [Train and Test data]")
print(pt_x_train.shape, pt_y_train.shape)
print(pt_x_test.shape, pt_y_test.shape)

print("\nTransfer Learning [Train and Test data]")
print(tl_x_train.shape, tl_y_train.shape)
print(tl_x_test.shape, tl_y_test.shape)




In [ ]:
model = Sequential()

model.add(Conv2D(32, 5, input_shape = (28, 28, 1), activation = 'relu'))
model.add(Conv2D(16, 5, activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Conv2D(8, 3, activation = 'relu'))

model.add(Flatten())
model.add(Dropout(0.4))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

In [ ]:


start = datetime.datetime.now()

model.fit(pt_x_train, pt_y_train,
         validation_data = (pt_x_test, pt_y_test),
         epochs = 10,
         shuffle = True,
         batch_size = 100,
         verbose = 1)

end = datetime.datetime.now()
print('\n Time taken for pre-train model: ', end - start)



In [ ]:
tl_model = Sequential(model.layers[:6])

tl_model.add(Dense(128, activation = 'relu'))
tl_model.add(Dense(10, activation = 'softmax'))

tl_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

tl_model.summary()

In [ ]:
start = datetime.datetime.now()

history = tl_model.fit(tl_x_train, tl_y_train,
            validation_data = (tl_x_test, tl_y_test),
            epochs = 10,
            shuffle = True,
            batch_size = 100,
            verbose = 2)

end = datetime.datetime.now()
print('\n Time taken for transfer learning model: ', end - start)